In [2]:
%pip install 
#패키지 설치, import
import pyproj

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gymdo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import folium
import pandas as pd
import numpy
from folium.plugins import MarkerCluster
#패키지 import

In [4]:
df = pd.read_csv('./data/편의시설/부산광역시_종합병원 현황_20211231.csv'
, encoding='UTF-8') #부산광역시 종합병원 현황 읽어오기 
hos_sub_df = df[df['도로명주소'].str.contains('기장군')]#관리기관명이 기장군인 것만 자름
print(hos_sub_df[['위도','경도','의료기관명']]) #기장군 병원 출력 테스트, 1개 존재


           위도          경도      의료기관명
27  35.321839  129.244334  동남권원자력의학원


In [63]:
df = pd.read_csv('./data/편의시설/fulldata_01_01_06_P_약국.csv'
, engine='python',encoding='cp949') #전국 약국 현황 읽어오기 
newDF=df.loc[df['소재지전체주소'].str.contains('기장군',na=False)]#기장만 끊어서
print(newDF['좌표정보(x)']) #좌표정보 테스트
newadd=newDF['사업장명'] #약국 이름만 따로 추출
proj_UMTK = pyproj.Proj('epsg:2097') #오래된 지리원 표준
proj_WGS84 = pyproj.Proj('epsg:4326') #경위도 좌표계
newPGS = pd.DataFrame({'위도':[],'경도':[]})#빈 데이터프레임 생성
for x1, y1 in zip(newDF['좌표정보(x)'],df['좌표정보(y)']):
    x2, y2 = pyproj.transform(proj_UMTK,proj_WGS84,y1,x1) #좌표 변환
    dataToInsert = pd.DataFrame({'위도':[x2],'경도':[y2]})#넣을 데이터
    newPGS = pd.concat([newPGS,dataToInsert],ignore_index=True)#newPGS에 데이터 넣기
newPGS = newPGS.replace([numpy.inf],numpy.nan)#inf nan으로 변환
newPGS=newPGS.dropna()#nan 제거
print(newPGS.head(10))#출력 테스트


1227     397529.796959
1358     396849.716710
7986     397942.700664
8091               NaN
8260               NaN
             ...      
39964    398054.092229
40209    403815.971500
42476    397354.244364
53051    391998.792141
53073    402168.434115
Name: 좌표정보(x), Length: 132, dtype: float64


C:\Users\gymdo\AppData\Local\Temp\ipykernel_11520\3013961560.py:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2, y2 = pyproj.transform(proj_UMTK,proj_WGS84,y1,x1) #좌표 변환


           위도          경도
1   35.153512  129.158131
2   35.144560  129.169885
5   35.167198  129.211508
6   35.146432  129.209162
7   35.167819  129.223502
8   35.153885  129.168545
9   35.153883  129.172944
10  35.166100  129.164774
11  35.154025  129.234579
12  35.167486  129.209746


In [62]:
kizang_pos= [35.23876594878238, 129.21575390899562] ## 기장읍 사무소
m = folium.Map(location=kizang_pos, zoom_start=17,width=750, 
               height=500)#지도 17의 줌으로 생성               
coords = hos_sub_df[['위도','경도','의료기관명']]#자료에서 위도, 경도, 경로당명 가져와서
coords2 = newPGS[['위도','경도']]
pgs_marker_cluster = MarkerCluster().add_to(m)
hos_marker_cluster = MarkerCluster().add_to(m)#MarkerCluster을 지도에 추가하고
for lat,long,tool in zip(coords['위도'],coords['경도'],coords['의료기관명']):#lat, long, tool을 이터레이터로 만들어서
    folium.Marker([lat,long], icon=folium.Icon(color='green'),tooltip=tool).add_to(hos_marker_cluster)#초록색 마커로 마우스 갖다대면 이름 보이게 표시
for lat, long  in zip(coords2['위도'],coords2['경도']):
    folium.Marker([lat,long],popup='약국입니다',icon = folium.Icon(color='red',icon='star'),tooltip='약국').add_to(pgs_marker_cluster)
m


ValueError: Location values cannot contain NaNs.